In [ ]:
# Download da biblioteca auxiliar

!wget https://github.com/ufrpe-2021-2-IA/music-dataset/releases/download/v1.1.2/mgd-1.1.2-py3-none-any.whl
%pip install mgd --find-links /content/

In [ ]:
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from mgd import utils

import pathlib

import numpy as np
from sklearn.base import is_classifier, clone
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

utils.download_experiments('raw')
utils.download_experiments('min_max')
utils.download_experiments('standardized')
base_dir = pathlib.Path('/content/').absolute()

In [ ]:
features_100 = ['mfcc_5_skew',
                'tonnetz_1_kurtosis',
                'tonnetz_3_kurtosis',
                'mfcc_4_kurtosis',
                'mfcc_11_skew',
                'tonnetz_5_kurtosis',
                'spectral_flatness_min',
                'mfcc_10_skew',
                'mfcc_7_skew',
                'mfcc_6_skew',
                'tonnetz_2_min',
                'mfcc_9_max',
                'spectral_flatness_skew',
                'spectral_flatness_median',
                'mfcc_8_min',
                'mfcc_4_min',
                'mfcc_1_std',
                'mfcc_6_min',
                'mfcc_8_std',
                'mfcc_10_mean',
                'mfcc_10_median',
                'mfcc_2_std',
                'mfcc_9_min',
                'mfcc_11_std',
                'mfcc_10_std',
                'spectral_roll_of_skew',
                'mfcc_10_max',
                'tonnetz_1_min',
                'mfcc_4_mean',
                'mfcc_4_median',
                'mfcc_9_std',
                'tonnetz_1_max',
                'mfcc_2_median',
                'tonnetz_2_max',
                'mfcc_12_std',
                'mfcc_2_mean',
                'mfcc_1_max',
                'mfcc_6_median',
                'mfcc_7_std',
                'mfcc_6_mean',
                'spectral_centroid_min',
                'mfcc_9_mean',
                'mfcc_11_min',
                'spectral_roll_of_min',
                'mfcc_9_median',
                'mfcc_12_max',
                'tonnetz_3_min',
                'tonnetz_4_min',
                'tonnetz_4_max',
                'mfcc_7_max',
                'spectral_flatness_mean',
                'mfcc_12_mean',
                'mfcc_12_median',
                'mfcc_11_mean',
                'mfcc_8_median',
                'tonnetz_3_max',
                'mfcc_11_median',
                'mfcc_8_mean',
                'mfcc_6_std',
                'tonnetz_6_max',
                'tonnetz_5_max',
                'tonnetz_5_min',
                'mfcc_4_std',
                'mfcc_4_max',
                'spectral_flatness_max',
                'mfcc_7_min',
                'mfcc_5_std',
                'tonnetz_6_min',
                'mfcc_5_min',
                'mfcc_8_max',
                'mfcc_3_std',
                'mfcc_6_max',
                'mfcc_5_median',
                'mfcc_7_mean',
                'mfcc_7_median',
                'mfcc_5_mean',
                'tonnetz_2_std',
                'spectral_flatness_std',
                'mfcc_2_max',
                'tonnetz_1_std',
                'spectral_roll_of_std',
                'mfcc_3_min',
                'tonnetz_3_std',
                'mfcc_1_median',
                'mfcc_1_mean',
                'mfcc_3_max',
                'mfcc_5_max',
                'tonnetz_4_std',
                'mfcc_3_median',
                'tonnetz_6_std',
                'mfcc_1_min',
                'mfcc_3_mean',
                'tonnetz_5_std',
                'spectral_centroid_median',
                'spectral_centroid_mean',
                'spectral_roll_of_median',
                'spectral_roll_of_mean',
                'spectral_centroid_std',
                'spectral_roll_of_max',
                'spectral_centroid_max']

features_all = None

# `KNN_baseline_all`

In [ ]:
# Carregando conjuntos de treino e testes
X_train, y_train = utils.prepare_sklearn(
    str(base_dir.joinpath('raw/train.csv')),
    features_all)
X_test, y_test = utils.prepare_sklearn(
    str(base_dir.joinpath('raw/test.csv')),
    features_all)

# Instanciação do GridSearch
knn_baseline_all = GridSearchCV(estimator=KNeighborsClassifier(),
                                param_grid={'n_neighbors': [3, 5, 8, 10, 15],
                                            'leaf_size': [20, 30, 40],
                                            'algorithm': ['auto', 'ball_tree', 'kd_tree'],
                                            'metric': ['minkowski', 'l1', 'l2', 'manhattan']},
                                cv=3,
                                refit=True)

# Buscar pelos parâmetros
knn_baseline_all.fit(X_train, Y_train)

# Obter melhor modelo
knn_baseline_all = knn_baseline_all.best_estimator_

# Fazer predição no conjunto de testes
predictions = knn_baseline_all.predict(X_test)

# Calcular métricas
classification_report(Y_val,
                      predictions,
                      digits=4,
                      output_dict=False)


# `SVM_minmax_100`

In [ ]:
# Carregando conjuntos de treino e testes
X_train, y_train = utils.prepare_sklearn(
    str(base_dir.joinpath('min_max/train.csv')),
    features_100)
X_test, y_test = utils.prepare_sklearn(
    str(base_dir.joinpath('min_max/test.csv')),
    features_100)

# Instanciação do GridSearch
svm_minmax_100 = GridSearchCV(estimator=svm.SVC(),
                                param_grid={'C': [0.25, 0.5, 1.0],
                                            'decision_function_shape': ['ovo', 'ovr'],
                                            'kernel': ['rbf', 'linear'],
                                            'max_iter': [200]},
                                cv=3,
                                refit=True)

# Buscar pelos parâmetros
svm_minmax_100.fit(X_train, Y_train)

# Obter melhor modelo
svm_minmax_100 = svm_minmax_100.best_estimator_

# Fazer predição no conjunto de testes
predictions = svm_minmax_100.predict(X_test)

# Calcular métricas
classification_report(Y_val,
                      predictions,
                      digits=4,
                      output_dict=False)


# `ANN_standardize_100`

In [ ]:
# Carregando conjuntos de treino e testes
X_train, y_train = utils.prepare_sklearn(
    str(base_dir.joinpath('standardized/train.csv')),
    features_100)
X_test, y_test = utils.prepare_sklearn(
    str(base_dir.joinpath('standardized/test.csv')),
    features_100)

# Instanciação do GridSearch
ann_standardize_100 = GridSearchCV(estimator=MLPClassifier(),
                                   param_grid={'hidden_layer_sizes': [(149, 40), (100, 40)],
                                               'activation': ['relu'],
                                               'solver': ['adam'],
                                               'alpha': [1e-4, 1e-5],
                                               'max_iter': [200],
                                               'learning_rate_init': [1e-3, 1e-4],
                                               'n_iter_no_change': [500]},
                                   cv=3,
                                   refit=True)

# Buscar pelos parâmetros
ann_standardize_100.fit(X_train, Y_train)

# Obter melhor modelo
ann_standardize_100 = ann_standardize_100.best_estimator_

# Fazer predição no conjunto de testes
predictions = ann_standardize_100.predict(X_test)

# Calcular métricas
classification_report(Y_val,
                      predictions,
                      digits=4,
                      output_dict=False)
